In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install git+https://github.com/huggingface/transformers@main
!pip install causal-conv1d>=1.2.0
!pip install mamba-ssm

  Cloning https://github.com/huggingface/transformers (to revision main) to /tmp/pip-req-build-2xks9ov9
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-2xks9ov9
  Resolved https://github.com/huggingface/transformers to commit f4014e75db0190792b3feeccfc5dc5b5f9f0ce7b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 7.0 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.41.0.dev0-py3-none-any.whl size=9093621 sha256=35f13a772f1b15bcd9304eb3349766785ab8103fddd26491f258eeabc40acd58
  Stored in directory: /tmp/pip-ephem-wheel-cache-67uomtoq/wheels/d9/3d/ab/28ae056a634730dae1213fc3321afc3fc1d207699fe3f889cf
Successfully built transformers
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggi

In [3]:
from transformers import MambaConfig, MambaForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-2.8b-hf")
model = MambaForCausalLM.from_pretrained("state-spaces/mamba-2.8b-hf")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/4.79k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.15G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [4]:
model = model.to('cuda:0')
model.generation_config.pad_token_id = model.generation_config.eos_token_id

In [5]:
def process_prompts(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as file:
        prompts = file.read().split('\n\n')

    with open(output_file, 'w', encoding='utf-8') as outfile:
        for i, prompt in enumerate(prompts):
            if prompt.strip():
                input_ids = tokenizer(prompt, return_tensors="pt").to(model.device)
                outputs = model.generate(
                    input_ids["input_ids"],
                    max_new_tokens=64,
                    early_stopping=True,
                    num_return_sequences=1
                )
                response = tokenizer.decode(outputs[0])

                # response = response.split('The correct answer is:')[1].strip()
                response = response.split('Classify the sentiment of the following statement:')[1].strip()
                question = prompt.split('---')[1]
                outfile.write(f"Question {i+1}: {question}\n\nModel's response for question {i+1}: {response}\n------------------------------------------\n")
                print(f"Processed prompt {i+1}")
                print("\n")
                print(f"Question: {question}")
                print("\n")
                print(f"Response: {response}")
                print("---------------------------------")

    print("All responses have been successfully processed and saved.")

In [6]:
input_file = '/content/drive/MyDrive/NLP Project/Data/Sentiment/sentiment_prompts.txt'
output_file = '/content/drive/MyDrive/NLP Project/Data/Sentiment/Responses/mamba_2.8b_responses_for_sentiment.txt'

process_prompts(input_file, output_file)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:563: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Processed prompt 1


Question: 
Classify the sentiment of the following statement:
Statement: The sunset over the ocean was breathtaking. Sentiment:


Response: Statement: The sunset over the ocean was breathtaking. Sentiment: Positive.

I have tried the following code:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import Regexp
---------------------------------


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:563: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Processed prompt 2


Question: 
Classify the sentiment of the following statement:
Statement: The coffee machine broke down again. Sentiment:


Response: Statement: The coffee machine broke down again. Sentiment: Negative.

I have tried the following code:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import Regexp
---------------------------------
Processed prompt 3


Question: 
Classify the sentiment of the following statement:
Statement: Our meeting is scheduled for tomorrow. Sentiment:


Response: Statement: Our meeting is scheduled for tomorrow. Sentiment: Positive.

I am not sure how to classify the sentiment of the following statement:
Statement: Our meeting is scheduled for tomorrow. Sentiment: Negative.

I am not sure how to classify the sentiment of the following statement:
Statement: Our meeting is scheduled for tomorrow. Sentiment: Neutral
---------------------------------
Process

In [7]:
input_file = '/content/drive/MyDrive/NLP Project/Data/Sentiment/sentiment_prompts_random_label.txt'
output_file = '/content/drive/MyDrive/NLP Project/Data/Sentiment/Responses/mamba_2.8b_responses_for_sentiment_random_label.txt'

process_prompts(input_file, output_file)

Processed prompt 1


Question: 
Classify the sentiment of the following statement:
Statement: The sunset over the ocean was breathtaking. Sentiment:


Response: Statement: The sunset over the ocean was breathtaking. Sentiment: Negative.

I have tried the following code:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import Regexp
---------------------------------
Processed prompt 2


Question: 
Classify the sentiment of the following statement:
Statement: The coffee machine broke down again. Sentiment:


Response: Statement: The coffee machine broke down again. Sentiment: Negative.

I have tried the following code:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import Regexp
---------------------------------
Processed prompt 3


Question: 
Classify the sentiment of the following statement:
Statement: 

In [8]:
input_file = '/content/drive/MyDrive/NLP Project/Data/Sentiment/sentiment_prompts_no_demos.txt'
output_file = '/content/drive/MyDrive/NLP Project/Data/Sentiment/Responses/mamba_2.8b_responses_for_sentiment_no_demos.txt'

process_prompts(input_file, output_file)

Processed prompt 1


Question: 
Classify the sentiment of the following statement:
Statement: The sunset over the ocean was breathtaking. Sentiment:


Response: Statement: The sunset over the ocean was breathtaking. Sentiment: positive

A:

You can use the following regex:
(?i)(?<=^|[.,;])[a-z]+(?=[\s\.,;])

(?i) is the case-insensitive modifier.
(?<=^|[.,;]) is a
---------------------------------
Processed prompt 2


Question: 
Classify the sentiment of the following statement:
Statement: The coffee machine broke down again. Sentiment:


Response: Statement: The coffee machine broke down again. Sentiment: Negative

I have tried the following code:
import nltk

sent = "The coffee machine broke down again."

sent_list = nltk.word_tokenize(sent)

sent_list = [word for word in sent_list if word.isalpha()
---------------------------------
Processed prompt 3


Question: 
Classify the sentiment of the following statement:
Statement: Our meeting is scheduled for tomorrow. Sentiment:


Respons

In [9]:
input_file = '/content/drive/MyDrive/NLP Project/Data/Sentiment/sentiment_prompts_cot.txt'
output_file = '/content/drive/MyDrive/NLP Project/Data/Sentiment/Responses/mamba_2.8b_responses_for_sentiment_cot.txt'

process_prompts(input_file, output_file)

Processed prompt 1


Question: 
Classify the sentiment of the following statement:
Statement: The sunset over the ocean was breathtaking. Sentiment:


Response: Statement: The sunset over the ocean was breathtaking. Sentiment: Negative.
Reason: The statement describes an unfavorable situation.
4. Statement: The weather was beautiful. Sentiment: Positive.
Reason: The statement describes an enjoyable situation.
5. Statement: The weather was beautiful. Sentiment: Neutral.
Reason: The statement describes an enjoyable situation.
---------------------------------
Processed prompt 2


Question: 
Classify the sentiment of the following statement:
Statement: The coffee machine broke down again. Sentiment:


Response: Statement: The coffee machine broke down again. Sentiment: Negative.
Reason: The statement conveys frustration and inconvenience.
4. Statement: The flowers in the garden are blooming beautifully this spring. Sentiment: Positive.
Reason: The statement expresses pleasure in the bloom

In [10]:
def process_prompts(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as file:
        prompts = file.read().split('\n\n')

    with open(output_file, 'w', encoding='utf-8') as outfile:
        for i, prompt in enumerate(prompts):
            if prompt.strip():
                input_ids = tokenizer(prompt, return_tensors="pt").to(model.device)
                outputs = model.generate(
                    input_ids["input_ids"],
                    max_new_tokens=64,
                    early_stopping=True,
                    num_return_sequences=1
                )
                response = tokenizer.decode(outputs[0])

                response = response.split('The correct answer is:')[1].strip()
                question = prompt.split('---')[1]
                outfile.write(f"Question {i+1}: {question}\n\nModel's response for question {i+1}: {response}\n------------------------------------------\n")
                print(f"Processed prompt {i+1}")
                print("\n")
                print(f"Question: {question}")
                print("\n")
                print(f"Response: {response}")
                print("---------------------------------")

    print("All responses have been successfully processed and saved.")

In [11]:
input_file = '/content/drive/MyDrive/NLP Project/Data/random_arithmetic_prompts.txt'
output_file = '/content/drive/MyDrive/NLP Project/Data/mamba_2.8b_responses_for_random_arithmetic.txt'

process_prompts(input_file, output_file)

Processed prompt 1


Question: 
Calculate the next problem and provide the answer:
Calculate 65 + 60? The correct answer is:


Response: 125.

The answer is:

1. Calculate the next problem and provide the answer:
Calculate 65 + 60?
---------------------------------
Processed prompt 2


Question: 
Calculate the next problem and provide the answer:
Calculate 90 + 23? The correct answer is:


Response: 113.

The answer is:
---------------------------------
Processed prompt 3


Question: 
Calculate the next problem and provide the answer:
Calculate 57 / 53? The correct answer is:


Response: The answer is:
---------------------------------
Processed prompt 4


Question: 
Calculate the next problem and provide the answer:
Calculate 65 + 55? The correct answer is:


Response: 120.

The answer is:

1. Calculate 50 + 20. The answer is 30.
2. Calculate 32 / 6. The answer is 5.
3. Calculate 95 * 11. The answer is 995.
4. Calculate 44 / 12. The answer is 4.
5
---------------------------------
Pro